<a href="https://colab.research.google.com/github/JulianGeis/forecasting_heatload/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
tf.test.gpu_device_name()
tf.compat.v1.disable_eager_execution()
import os
import numpy as np
from math import sqrt
from numpy import concatenate
import datetime as dt
import matplotlib.pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import pandas as pd
import random as rd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers import MaxPooling1D
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.optimizers import Adagrad
from tensorflow.python.keras.optimizers import Adadelta
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.initializers import RandomUniform

from google.colab import files
uploaded = files.upload()

### methods

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df[0].shift(-i))
        if i == 0:
            names += [('var1(t)')]
        else:
            names += [('var1(t+%d)' % (i))]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

Saving data_dummies_index to data_dummies_index


In [0]:
# Observing script runtime
startScript = dt.datetime.now()  # observe script running time

optimising = 1  # default model vs. hyper-parameter optimising
runs = 2
result = pd.DataFrame(index=range(runs), columns= ['rmse_all','rmse_val','rmse_fit','train_days', 'batch_size','train_algo','lr', 'decay','patience', 'val_split', 'activation',
                                                   'duration','scaling','regularisation', 'dropout', 'epochs'])

for i in range(0,runs,1):
  print('run: ' +str(i+1))
  
  data = pd.read_csv('data_dummies_index', header=0, index_col=0, date_parser=pd.to_datetime)
  data['temp']= data['temp'].shift(-72)    # shift temperature 72h in past to obtain 3 day ahead temperature forecast (temp from 2014-01-04 00:00:00 is switched to 2014-01-01 00:00:00
  data.rename(columns={'temp':'temp_72'}, inplace=True)
  
  # Observing runtime of single net
  startNet = dt.datetime.now()  # observe script running time
  
  if (optimising == 0):
    scaling = 2   # 0 = No scaling | 1 = MinMax Scaling | 2 = Z-Score 
    preparation = 1
    train_days = 5
    train_algo = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    val_split = 0.25
    activation = 'sigmoid'
    patience = 20
    batch_size = 200
    decay = 0   # new learning rate = old learning rate / (1 + decay * iterations)) with iterations are the number of individual algo stepts not epochs
    regularisation = 0
    dropout = 0.1
    
  if (optimising == 1):
    
    # Hyper-parameter optimising:   
    lr_sgd_rms = rd.choice([0.0001, 0.001, 0.01, 0.1]) # default = 0.001
    lr_adagrad = rd.choice([0.001, 0.01, 0.1, 1])   # default = 0.01
    lr_adam = rd.choice([0.00001, 0.0001, 0.001, 0.01]) # default = 0.0001
    decay = rd.choice([0,0.0001,0.001, 0.01])   # rate of decay for the learning rate (next epoch will be started using the current learning rate divided by the decay parameter)

    # optimizers:
    sgd=SGD(lr=lr_sgd_rms, momentum=0.9, decay=decay, nesterov=False)  # default: SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
    rmsprop=RMSprop(lr=lr_sgd_rms, rho=0.9, epsilon=None, decay=decay) # default:RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
    adagrad=Adagrad(lr=lr_adagrad, epsilon=None, decay=decay)    # default: Adagrad(lr=0.01, epsilon=None, decay=0.0)
    adam=Adam(lr=lr_adam, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay, amsgrad=False)   # default=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    train_algo = adam #  rd.choice([sgd, rmsprop, adagrad, adam])
    
    scaling = 2   # rd.choice([0, 1, 2])   # 0 = No scaling | 1 = MinMax Scaling | 2 = Z-Score 
    preparation = 1 # rd.choice([1, 2])
    val_split = rd.choice([0.25, 0.3, 0.35])
    train_days = rd.choice([ 5, 7, 9])
    activation = 'relu' # rd.choice([ 'sigmoid', 'relu', 'tanh'])
    patience = rd.choice([ 10, 20])
    batch_size = rd.choice([50, 100, 150, 200, 250, 300])
    regularisation = rd.choice([0, 0.001, 0.01, 0.1])
    dropout = rd.choice([0.1, 0.2 , 0.3])

  # 1.3 Scaling

  # Scaling the data between min and max: x_scaled = (x - min(x)) / ( max(x) - min(x)) only with parameters calculated from the train set
  if (scaling == 1):
      min, max = -1, 1
      min_load_train, max_load_train = np.max(data['load']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), np.min(data['load']['2014-01-01 00:00:00':'2015-12-31 23:00:00'])
      data['load'] = ((data['load'] - min_load_train) / (max_load_train - min_load_train)) * (max - min) + min    # Inverse scaling: x = (x_scaled - min) / (max - min) * (max_load_train - min_load_train) + min_load_train
      min_temp_train, max_temp_train = np.max(data['temp_72']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), np.min( data['temp_72']['2014-01-01 00:00:00':'2015-12-31 23:00:00'])
      data['temp_72'] = ((data['temp_72'] - min_temp_train) / (max_temp_train - min_temp_train)) * (max - min) + min  # Inverse scaling: x = (x_scaled - min) / (max - min) * (max_load_train - min_load_train) + min_load_train

  # Scaling the data as folllows:  x_scaled = (x - mean(x)) / st(x) only with parameters calculated from the train set
  if (scaling == 2):
      mean_load_train, sd_load_train = np.mean(data['load']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), np.sqrt(np.var(data['load']['2014-01-01 00:00:00':'2015-12-31 23:00:00']))
      data['load'] = (data['load'] - mean_load_train) / sd_load_train   # Inverse scaling: x = x_scaled * sd_load_train + mean_load_train
      mean_temp_train, sd_temp_train = np.mean(data['temp_72']['2014-01-01 00:00:00':'2015-12-31 23:00:00']), np.sqrt(np.var(data['temp_72']['2014-01-01 00:00:00':'2015-12-31 23:00:00']))
      data['temp_72'] = (data['temp_72'] - mean_temp_train) / sd_temp_train  # Inverse scaling: x = x_scaled * sd_load_train + mean_load_train


  data = data[['load','temp_72']]       # chose columns to include in forecast (26304, 3) from 2014-01-01 00:00:00 until 2016-12-31 23:00:00

  # ensure all data is float
  values = data.values.astype('float32')

  # specify the number of lag hours
  train_n = 24*train_days
  predict_n = 24*3
  n_features = data.shape[1]   # number of input variables


  # split into train and test sets
  train_samples = round(365*2*24*(1-val_split))
  val_samples = 365*2*24
  train = series_to_supervised(values[:train_samples, :],train_n, predict_n).values
  val = series_to_supervised(values[train_samples:val_samples, :],train_n, predict_n).values
  test = series_to_supervised(values[val_samples:, :],train_n, predict_n).values

  # split into input and outputs
  input_cols = train_n * n_features    # columns of input tensor
  X_train, y_train = train[:, :input_cols], train[:, -predict_n:]  # (training samples, columns of input tensor) | (training samples, number of periods to predict)
  X_val, y_val = val[:, :input_cols], val[:, -predict_n:]  # (training samples, columns of input tensor) | (training samples, number of periods to predict)
  X_test, y_test = test[:, :input_cols], test[:, -predict_n:]  # (testing samples, columns of input tensor) | (testing samples samples, number of periods to predict)


  # Selecting only every j-th sample
  j = 1   # j = 24 means you only consider every 24th sample
  X_train, y_train = np.array(pd.DataFrame(X_train).iloc[::j, :]), np.array(pd.DataFrame(y_train).iloc[::j, :])
  X_test, y_test = np.array(pd.DataFrame(X_test).iloc[::j, :]), np.array(pd.DataFrame(y_test).iloc[::j, :])

  # reshape input to be 3D [samples, timesteps, features]
  X_train = X_train.reshape((X_train.shape[0], train_n, n_features))  # (training samples, number of periods to train, number of input variables)
  X_test = X_test.reshape((X_test.shape[0], train_n, n_features)) # # (testing samples, number of periods to train, number of input variables)
  X_val = X_val.reshape((X_val.shape[0], train_n, n_features))  # (training samples, number of periods to train, number of input variables)

  ##### 02 Network selection
  
  ##### 02 Network selection
  hiddenNeurons = 160

  # architecture
  mv_lstm = Sequential()
  mv_lstm.add(LSTM(units=hiddenNeurons, activation=activation, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True, 
                   recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='normal', recurrent_initializer='orthogonal', 
                   bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None,
                   activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, 
                   recurrent_dropout=0.0, implementation=1, return_state=False, go_backwards=False, stateful=False, unroll=False))
  mv_lstm.add(Dropout(0.2))

  mv_lstm.add(LSTM(units=hiddenNeurons, activation=activation, return_sequences=False, 
                   recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='normal', recurrent_initializer='orthogonal', 
                   bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None,
                   activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, 
                   recurrent_dropout=0.0, implementation=1, return_state=False, go_backwards=False, stateful=False, unroll=False))
  mv_lstm.add(Dropout(0.2))

  mv_lstm.add(Dense(units=predict_n, activation='linear', kernel_initializer=RandomUniform(minval=-0.05, maxval=0.05)))

  # compilation
  mv_lstm.compile(optimizer=train_algo, loss='mse') 

  # patient early stopping
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)

  # fit model
  history = mv_lstm.fit(X_train, y_train, batch_size = batch_size, epochs=100, verbose=2, 
                        validation_data=(X_val, y_val),callbacks=[es], shuffle=True)

  # prediction
  if (scaling == 0):
      yhat = mv_lstm.predict(X_test)
      yfit = mv_lstm.predict(X_train)  # fitted values
      yval = mv_lstm.predict(X_val)  # fitted values

  if (scaling == 1):
      yhat_s = mv_lstm.predict(X_test)
      yhat = (yhat_s - min) / (max - min) * (max_load_train - min_load_train) + min_load_train
      y_test = (y_test - min) / (max - min) * (max_load_train - min_load_train) + min_load_train

      yval_s = mv_lstm.predict(X_val)
      yval = (yval_s - min) / (max - min) * (max_load_train - min_load_train) + min_load_train
      y_val = (y_val - min) / (max - min) * (max_load_train - min_load_train) + min_load_train

      yfit_s = mv_lstm.predict(X_train) 
      yfit = (yfit_s - min) / (max - min) * (max_load_train - min_load_train) + min_load_train
      y_train = (y_train - min) / (max - min) * (max_load_train - min_load_train) + min_load_train

  if (scaling == 2):
      yhat_s = mv_lstm.predict(X_test)
      yhat = yhat_s * sd_load_train + mean_load_train
      y_test = y_test * sd_load_train + mean_load_train

      yfit_s = mv_lstm.predict(X_train)
      yfit = yfit_s * sd_load_train + mean_load_train
      y_train = y_train  * sd_load_train + mean_load_train

      yval_s = mv_lstm.predict(X_val)
      yval = yval_s * sd_load_train + mean_load_train
      y_val = y_val  * sd_load_train + mean_load_train



  # Specification
  print('Optimising: ' + str(optimising))
  
  # evaluation measures
  rmse_all = np.sqrt(mean_squared_error(y_test,yhat)); print('rmse_all: ' + str(rmse_all))
  rmse_val = np.sqrt(mean_squared_error(y_val,yval)); print('rmse_val: ' + str(rmse_val))
  rmse_fit = np.sqrt(mean_squared_error(y_train,yfit)); print('rmse_fit: ' + str(rmse_fit))
  time_net = dt.datetime.now() - startNet; print ('time_net: ' + str(time_net))  # observe runtime of single neural net
  
  if (optimising == 1):
    result['rmse_all'][i] = rmse_all
    result['rmse_val'][i] = rmse_val
    result['rmse_fit'][i] = rmse_fit
    result['train_days'][i] = train_days
    result['batch_size'][i] = batch_size
    if(train_algo == sgd ):
      algo = 'sgd'
    if(train_algo == rmsprop):
      algo = 'rmsprop'
    if(train_algo == adagrad):
      algo = 'adagrad'
    if(train_algo == adam):
      algo ='adam'
    result['train_algo'][i] = algo    
    if(train_algo == sgd or train_algo == rmsprop):
      lr = lr_sgd_rms
    if(train_algo == adagrad):
      lr = lr_adagrad
    if(train_algo == adam):
      lr = lr_adam
    result['lr'][i] = lr
    result['decay'][i] = decay
    result['patience'][i] = patience 
    result['val_split'][i] = val_split
    result['activation'][i] = activation   
    result['duration'][i] = time_net 
    result['scaling'][i] = scaling
    result['regularisation'][i] = regularisation 
    result['dropout'][i] = dropout
    result['epochs'][i] = es.stopped_epoch
    result

time_script = dt.datetime.now() - startScript; print ('time_script: ' + str(time_script))  # observe runtime of whole script

In [0]:
##### 03 Evaluation

# plot history
plt.plot(history.history['loss'][10:], label='train')
plt.plot(history.history['val_loss'][10:], label='test')
plt.legend()

In [0]:
# plot prediction
 for i in range(0, len(y_test), 24*7*4):
     plt.figure(i)
     plt.plot(yhat[i], label='prediction')
     plt.plot(y_test[i], label ='observation')
     plt.title('prediction of week ' + str((round(i/(24*7)+1,0))))
     plt.xlabel('hour');plt.ylabel('MW');plt.legend()
     plt.legend()

In [0]:
# plot "fitted values"
 for i in range(0, len(y_train), 24*7*4):
     plt.figure(i+10)
     plt.plot(yfit[i], label='fitted values')
     plt.plot(y_train[i], label ='observation')
     plt.title('fitted values of week ' + str((round(i/(24*7)+1,0))))
     plt.xlabel('hour');plt.ylabel('MW');plt.legend()
     plt.legend()